# **Prompt Tuning for Sequence Classificaiton**

In this example we will be fine-tuning the model *RoBERTa Large* to classify a sequence of tokens. For this purpose, we will use a PEFT method called **Prompt Tuning**, which prepends a trainable embedding matrix to the input embeddings. We will use **transformers** to download models and training, **datasets** for data downdload **peft** for Prompt Tuning model initialization, **evaluate** for loading evaluation metrics and **wandb** (Weights & Biases) to log the results.

You can also open this example in google colab:

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Wicwik/peft_tutorial/blob/main/examples/pt_classification.ipynb)

### **0. Install and import required modules**

In [ ]:
# 4.35.2 for compatibility with adapters
%pip install -q --user transformers==4.35.2
%pip install -q --user datasets
%pip install -q --user peft
%pip install -q --user evaluate
%pip install -q --user wandb

In [ ]:
import torch
import wandb
import evaluate

from peft import (
    get_peft_model,
    PromptTuningConfig,
    TaskType,
    PromptTuningInit,
)
from transformers import ( 
    AutoModelForSequenceClassification, 
    AutoTokenizer, 
    TrainingArguments,
    Trainer,
    default_data_collator
)

from datasets import load_dataset

### **1. Set variables**

We will be fine-tuning the pre-trained version of model [roberta-large](https://huggingface.co/FacebookAI/roberta-large) which has **355M** parameters. We will set the max **input length to 128** tokens and train for **3 epochs** with **batch size of 32**.

In [ ]:
device = "cuda"
model_name_or_path = "roberta-large"
tokenizer_name_or_path = "roberta-large"

max_length = 128
lr = 1e-3
num_epochs = 3
batch_size = 32 # in case of "unable to allocate" errors, decrease batch size to some lower number (e.g. 8,16) 

### **2. Create PEFT model**

Next we will create the PEFT model. The Hugging Face PEFT module will freeze the weights and add prompt encoder automatically.

Compare the model architecutres with and without the added prompt encoder weights.

In [ ]:
peft_config = PromptTuningConfig(task_type=TaskType.SEQ_CLS, num_virtual_tokens=10, prompt_tuning_init=PromptTuningInit.TEXT, tokenizer_name_or_path=tokenizer_name_or_path, prompt_tuning_init_text="Is the meaning of these sentences equivalent:")


model = AutoModelForSequenceClassification.from_pretrained(model_name_or_path, return_dict=True)

# comment next 2 lines if you want to do FFT
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()
model

We can also see that we have been able to reduce the number of trainable parameters to mere **0.2% of original model parameters**.

### **3. Dataset and preprocessing**

In [ ]:
# we have also a usable test split already, so we don't need to make it
dataset = load_dataset("glue", "mrpc")
dataset["train"][0]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

def preprocess_function(examples):
    model_inputs = tokenizer(examples["sentence1"], examples["sentence2"], max_length=max_length, padding="max_length", truncation=True, return_tensors="pt")
    return model_inputs

processed_datasets = dataset.map(
    preprocess_function,
    batched=True,
    num_proc=1,
    remove_columns=["sentence1", "sentence2", "idx"],
    load_from_cache_file=False,
    desc="Running tokenizer on dataset",
)

processed_datasets = processed_datasets.rename_column("label", "labels")

train_dataset = processed_datasets["train"].shuffle()
eval_dataset = processed_datasets["validation"]
test_dataset = processed_datasets["test"]

### **4. Training and evaluation**

In [ ]:
metric = evaluate.load("glue", "mrpc")

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    preds = preds.argmax(axis=1)

    return metric.compute(predictions=preds, references=labels)

training_args = TrainingArguments(
    "out",
    per_device_train_batch_size=batch_size,
    learning_rate=lr,
    num_train_epochs=num_epochs,
    evaluation_strategy="epoch",
    logging_strategy="epoch",
    save_strategy="no",
)

In [ ]:

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=default_data_collator,
    compute_metrics=compute_metrics,
)
trainer.train()

trainer.evaluate(eval_dataset=test_dataset, metric_key_prefix="test")

if wandb.run is not None:
    wandb.finish()

### **5. Save and load**

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

peft_model_id = f"{model_name_or_path}_{peft_config.peft_type}_{peft_config.task_type}"
model.save_pretrained(peft_model_id)

ckpt = f"{peft_model_id}/adapter_model.safetensors"
!du -h $ckpt

In [ ]:
from peft import PeftModel, PeftConfig

peft_model_id = f"{model_name_or_path}_{peft_config.peft_type}_{peft_config.task_type}"

config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForSequenceClassification.from_pretrained(config.base_model_name_or_path)
model = PeftModel.from_pretrained(model, peft_model_id)

In [ ]:
inputs = tokenizer("this is an apple", "this is a fruit", return_tensors="pt")
print(inputs)
with torch.no_grad():
    outputs = model(**inputs)
    print(outputs.logits.argmax(axis=1))